In [ ]:
from PosSelect_Functions_Old import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pandas as pd
import numpy as np
import copy
import seaborn as sns
from scipy.stats import mannwhitneyu as mwu
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import wilcoxon
from scipy.optimize import curve_fit
from scipy.stats import fisher_exact

hfont = {'fontname':'Arial'}
plt.rcParams["font.family"] = "Arial"

#Code borrowed heavily from here: https://stackoverflow.com/questions/62375034/find-non-overlapping-area-between-two-kde-plots
plt.rcParams.update(
    {"text.usetex": False}
)

#Function to quantile normalize PhyloP scores
def quantile_normalize_phylop(v, vv):
    #Upsample the lower one, maximum of one duplicate per entry added
    v_samp = v.sample(frac = (vv.shape[0] - v.shape[0])/v.shape[0], replace = True)
    v_samp.index = range(v.shape[0], v.shape[0] + v_samp.shape[0])
    
    v_new = pd.concat([v, v_samp]).sort_values("PhyloP447")

    #Make new dataframes and join
    vv_new = vv.copy().sort_values("PhyloP447")
    #v_new.columns = ["v0", "v1", "v2", "v3", "v4"]
    #vv_new.columns = ["vv0", "vv1", "vv2", "vv3"]
    v_new.index = list(range(v_new.shape[0]))
    vv_new.index = list(range(vv_new.shape[0]))
    v_new.columns = [x + " poly" for x in v_new.columns]
    vv_new.columns = [x + " fixed" for x in vv_new.columns]
    v_quant = v_new.join(vv_new)

    #Make dataframe to normalize
    df = v_quant[["PhyloP447 poly", "PhyloP447 fixed"]].copy()
    rank_mean = df.stack().groupby(df.rank(method='first').stack().astype(int)).mean()
    print(rank_mean)
    df = df.rank(method='min').stack().astype(int).map(rank_mean).unstack()
    df.columns = ["PhyloP447 poly", "PhyloP447 fixed"]
    new_v1 = np.mean(df, axis = 1)
    df["Statistic"] = new_v1
    df = df[["Statistic"]].copy()
    df = df.join(v_quant)
    return df

hpo = pd.read_csv("../DPSC_CNCC/HPO_AccelEvol_Input.txt", sep= "\t")
d_HPO = {}

for index, row in hpo.iterrows():
    d_HPO[row["Term"]] = row["Genes"].split(";")

In [ ]:
#Read in the data
v, yvalls = read_missense(path = "./", maf_cut = 0.1)
yvalls = unfold_missense(yvalls)


In [ ]:
#Perform quantile normalization
vv_mis2 = vv_mis[(vv_mis["MAFMaxAcrossAncestry"] > 0.1) & (vv_mis["MAFMaxAcrossAncestry"] < 0.9)]
qm = quantile_normalize_phylop(vv_mis2, v_mis)

fixed_cols = ["Statistic"]
poly_cols = ["Statistic"]
for c in qm.columns:
    if "fixed" in c:
        fixed_cols.append(c)
    elif "poly" in c:
        poly_cols.append(c)
qm_for_fixed = qm[fixed_cols]
qm_for_poly = qm[poly_cols]
qm_for_poly = qm_for_poly.sample(frac = 1, replace = False).drop_duplicates(["Position poly"])

qm_for_poly.columns = [x.replace(" poly", "") for x in qm_for_poly.columns]
qm_for_fixed.columns = [x.replace(" fixed", "") for x in qm_for_fixed.columns]

In [ ]:
sns.kdeplot({"Fixed":qm_for_fixed["Statistic"], "Normed":qm_for_poly["Statistic"]}, common_norm = False)

In [ ]:
#Test on HPO gene sets
c = 0
out = []
maf_cut = 0.1

qm_for_fixed.index = qm_for_fixed["Gene"]
qm_for_poly.index = qm_for_poly["Gene"]


for key in d_HPO.keys():
    if c%1000 == 0:
        print(c)
    if len(d_HPO[key]) >= 20 and len(d_HPO[key]) < 100:
        vk = qm_for_fixed.loc[np.intersect1d(qm_for_fixed.index, d_HPO[key])]
        vvk = qm_for_poly.loc[np.intersect1d(qm_for_poly.index, d_HPO[key])]
        x2 = [np.float64(x) for x in list(vk["Statistic"])]
        yvals2 = [np.float64(j) for j in list(vvk["Statistic"])]
        yvals2.sort()
        if len(vk.index) >= 50 and len(vvk.index) >= 30:
            vvv = prepare_alpha(vk, vvk, stat = "Statistic")
            cuttt = 0.6
            cutoff = yvals2[int(floor((len(yvals2)*cuttt)))]
            
            alpha = compute_alpha_cutoff(vvv, dn_cut = 0.01, plot = False, cutoff = cutoff)
            table1 = alpha[2]
            table2 = alpha[3]

            p = (fisher_exact(table1, alternative = "greater")[1] + fisher_exact(table2, alternative = "greater")[1])/2
            out.append([key, np.median(x2), len(x2), np.median(yvals2), len(yvals2), mwu(x2, yvals2, alternative = "greater")[1], p, alpha[0], alpha[1], table1, table2])
    c += 1
    
from statsmodels.stats.multitest import fdrcorrection
df = pd.DataFrame(out)
df = df.sort_values(5)
df["MWU QM FDR"] = fdrcorrection(df[5])[1]

df.columns = ["Term", "Median Fixed PhyloP", "Number Fixed Variants", "Median Polymorphic PhyloP", "Number Polymorphic Variants", "MWU p-value", "Fisher exact p-value", "alpha", "Cutoff", "[[dc1, pc1], [du1, pu1]]", "[[dc2, pc2], [du2, pu2]]", "FDR MWU"]
df.to_csv("ForPub_NewGOHPO_FisherExact_Mis_QuantNorm_PhyloP_MafCut_AF" + str(maf_cut) + ".csv", index = False)

In [ ]:
#Try without quantile normalization
c = 0
out = []
v.index = v["Gene"]
yvalls.index = yvalls["Gene"]
maf_cut = 0.1
for key in d_HPO.keys():
    if c%1000 == 0:
        print(c)
    if len(d_HPO[key]) >= 15 and len(d_HPO[key]) < 100:
        vk = v.loc[np.intersect1d(v.index, d_HPO[key])]
        vvk = yvalls.loc[np.intersect1d(yvalls.index, d_HPO[key])]
        x2 = [np.float64(x) for x in list(vk["PhyloP447"])]
        yvals2 = [np.float64(j) for j in list(vvk["PhyloP447"])]
        yvals2.sort()
        if len(vk.index) >= 50 and len(vvk.index) >= 30:
            cuttt = 0.6
            vvv = prepare_alpha(vk, vvk, stat = "PhyloP447")
            cutoff = yvals2[int(floor((len(yvals2)*cuttt)))]

            alpha = compute_alpha_cutoff(vvv, dn_cut = 0.01, plot = False, cutoff = cutoff)
            table1 = alpha[2]
            table2 = alpha[3]
            out.append([key, np.median(x2), len(x2), np.median(yvals2), len(yvals2), mwu(x2, yvals2, alternative = "greater")[1], (fisher_exact(table1, alternative = "greater")[1] + fisher_exact(table2, alternative = "greater")[1])/2, alpha[0], alpha[1], table1, table2])
            if key == "Cardiac arrest":
                print(cutoff)
                print(alpha)
    c += 1
    
from statsmodels.stats.multitest import fdrcorrection
df = pd.DataFrame(out)
df = df.sort_values(5)
df["FDR MWU"] = fdrcorrection(df[5])[1]

df.columns = ["Term", "Median Fixed PhyloP", "Number Fixed Variants", "Median Polymorphic PhyloP", "Number Polymorphic Variants", "MWU p-value", "Fisher exact p-value", "alpha", "Cutoff", "[[dc1, pc1], [du1, pu1]]", "[[dc2, pc2], [du2, pu2]]", "FDR MWU"]
df.to_csv("ForPub_FisherExact_Cuttt0.6_Unfold_NewHPO_Mis_PhyloP_MafCut_AF" + str(maf_cut) + ".csv", index = False)

In [ ]:
#Make plot for cardiac arrest
vk = v_mis[v_mis["Gene"].isin(d_HPO["Cardiac arrest"])]
vvk = vv_mis[vv_mis["Gene"].isin(d_HPO["Cardiac arrest"])]

vvk = vvk[(vvk["MAFMaxAcrossAncestry"] > 0.1) & (vvk["MAFMaxAcrossAncestry"] < 0.9)]

vsk = vs[vs["Gene"].isin(d_HPO["Cardiac arrest"])]
vvsk = vvs2[vvs2["Gene"].isin(d_HPO["Cardiac arrest"])]

compute_alpha_cutoff(prepare_alpha(vk, vvk), plot = True, title = "Cardiac arrest genes")

In [ ]:
#Read in constraint and synaptic data
pli = pd.read_csv("gnomad.v4.1.constraint_metrics.tsv", sep = "\t")
pli = pli.dropna()
pli = pli[pli["lof.pLI"] > 0.9]
keep_pli = pli["gene"]

syngo = pd.read_csv("../Cell_Type_Prop/Cell_Type_Prop/syngo_genes.csv")
syngo = syngo["hgnc_symbol"]


In [ ]:
#Read in nonsyn data
v_mis, vv_mis = read_missense(path = "./", maf_cut = 0.01)
vv_mis = unfold_missense(vv_mis)
vv_mis2 = vv_mis[(vv_mis["UnfoldedMAF"] > 0.5) & (vv_mis["UnfoldedMAF"] < 0.9)]

In [ ]:
#Read in syn data
vs, vvs = read_syn(maf_cut = 0.1)
vvs = unfold_syn(vvs)
vvs2 = vvs[(vvs["UnfoldedMAF"] > 0.5) & (vvs["UnfoldedMAF"] < 0.9)]

In [ ]:
#Isolate mucin genes, tested removing these to make sure they didn't affect results
muc = []
for i in v_mis["Gene"]:
    if i.startswith("MUC"):
        muc.append(i)
for i in vv_mis["Gene"]:
    if i.startswith("MUC"):
        muc.append(i)
muc = list(set(muc))


In [ ]:
#Little evidence for positive selection overall
vk = v_mis.copy()
vvk = vv_mis2.copy()

#vk = vk[~vk['Gene'].isin(muc)]
#vvk = vvk[~vvk['Gene'].isin(muc)]

cuttt = 0.6

z = list(vvk["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]


vvv = prepare_alpha(vk, vvk)
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "All genes", window = [-5, 12])
print(alpha)
print((fisher_exact(alpha[-2])[0] + fisher_exact(alpha[-3])[0])/2)

In [ ]:
#Stronger positive selection on haploinsufficient genes
vk = v_mis[v_mis["Gene"].isin(keep_pli)].copy()
vvk = vv_mis2[vv_mis2["Gene"].isin(keep_pli)].copy()

#vk = vk[~vk['Gene'].isin(muc)]
#vvk = vvk[~vvk['Gene'].isin(muc)]

cuttt = 0.6

z = list(vvk["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]


vvv = prepare_alpha(vk, vvk)
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Haploinsufficient genes")
print(alpha)
print((fisher_exact(alpha[-2])[1] + fisher_exact(alpha[-3])[1])/2)

In [ ]:
#Can adjust to 2 or 4 depending on what we want to test
low_mis = pli[pli["mis.z_score"] <= 2]["gene"]
high_mis = pli[pli["mis.z_score"] > 4]["gene"]

vk2 = vk[vk["Gene"].isin(high_mis)]
vvk2 = vvk[vvk["Gene"].isin(high_mis)]
vvv = prepare_alpha(vk2, vvk2)

cuttt = 0.6

z = list(vvk2["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]

alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "HI, very nonsynonymous-depleted")
print(alpha)
print((fisher_exact(alpha[-2])[1] + fisher_exact(alpha[-3])[1])/2)


In [ ]:
#Took these genes and imported to GOrilla, copied and pasted the 48 NeuronProjection genes into "NeuronProjection4.csv"
#Did the same copy paste procedure with IonChannel genes
vk.to_csv("AllHI_WithMis.csv")
vvk.to_csv("AllHI_WithMis_Poly.csv")
vk[vk["Gene"].isin(high_mis)].to_csv("MisDepHI_WithMis.csv")
vvk[vvk["Gene"].isin(high_mis)].to_csv("MisDepHI_WithMis_Poly.csv")

In [ ]:
#Read in genes from GOrilla
z = pd.read_csv("NeuronProjection2_wPoly.csv", header = None)
cc = []
for i in z[0]:
    cc.append(i.split(" - ")[0])

In [ ]:
#Do MK test for supplemental text results
vvs2 = vvs[(vvs["UnfoldedMAF"] > 0.5) & (vvs["UnfoldedMAF"] < 0.9)]
vv_mis2 = vv_mis[(vv_mis["UnfoldedMAF"] > 0.5) & (vv_mis["UnfoldedMAF"] < 0.9)]

#cc = list(np.unique(list(pd.read_csv("VeryMisDepHI_WithMis_Poly.csv")["Gene"]) + list(pd.read_csv("VeryMisDepHI_WithMis.csv")["Gene"])))

synp = vvs2[vvs2['Gene'].isin(np.intersect1d(high_mis, cc))].shape[0]
synf = vs[vs['Gene'].isin(np.intersect1d(high_mis, cc))].shape[0]

misp = vv_mis2[vv_mis2['Gene'].isin(np.intersect1d(high_mis, cc))].shape[0]
misf = v_mis[v_mis['Gene'].isin(np.intersect1d(high_mis, cc))].shape[0]

#synp = vvs2[vvs2['Gene'].isin(cc)].shape[0]
#synf = vs[vs['Gene'].isin(cc)].shape[0]

#misp = vv_mis2[vv_mis2['Gene'].isin(cc)].shape[0]
#misf = v_mis[v_mis['Gene'].isin(cc)].shape[0]

#synp = vvs2[vvs2['Gene'].isin(high_mis)].shape[0]
#synf = vs[vs['Gene'].isin(high_mis)].shape[0]

#misp = vv_mis2[vv_mis2['Gene'].isin(high_mis)].shape[0]
#misf = v_mis[v_mis['Gene'].isin(high_mis)].shape[0]


print(fisher_exact([[misf, synf], [misp, synp]], alternative = "greater"))
print([[misf, synf], [misp, synp]])
vvv = prepare_alpha(v_mis[v_mis['Gene'].isin(cc)], vv_mis2[vv_mis2['Gene'].isin(cc)])
compute_alpha_new(vvv, plot = True)



In [ ]:
#Getting confidence intervals in R since it is a complete pain in the ass in python
#Alternative is greater so only showing lower bound
#order is OR, lower bound
ci_05_proj_02 = [1.646604, 1.031718]
ci_025_proj_02 = [1.640916, 1.19403]
#ci_05_ion_02 = []
#ci_025_ion_02 = []
ci_05_all_02 = [0.9844314, 0.8549695]
ci_025_all_02 = [1.138728, 1.024264]

ci_05_proj_04 = [2.329714, 1.117055]
ci_025_proj_04 = [1.91541, 1.194166]
#ci_05_ion_04 = []
#ci_025_ion_04 = []
ci_05_all_04 = [1.067388, 0.8189038]
ci_025_all_04 = [1.184932, 0.9725502]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
categories = ['All other genes', 'Neuron projection genes']
values = [ci_05_all_04[0], ci_05_proj_04[0]]
lower_ci = [ci_05_all_04[0] - ci_05_all_04[1], ci_05_proj_04[0] - ci_05_proj_04[1]]  # Lower confidence intervals

# Create the bar plot
plt.bar(categories, values, color=["#40A94D", "#F42FF5"], label='Values', alpha = 0.8)

# Add lower confidence intervals as error bars
for i, (x, y, ci) in enumerate(zip(categories, values, lower_ci)):
    plt.vlines(x, y - ci, y, color='black', label='Lower CI' if i == 0 else "")
    plt.hlines(y - ci, i-0.125, i+0.125, color='black')
    #if i == 0:
    #    plt.text(i - 0.025, y-0.025, "*", size = 20)

# Customize the plot
plt.xlabel('Gene set', size = 16)
plt.ylabel('Odds ratio', size = 16)
plt.title('MK test on very nonsynonymous-depleted genes', size = 16)
plt.legend([], [], frameon = False)
plt.xticks(size = 14)
plt.show()



In [ ]:
#Testing for positive selection using sign test on cortical organoid data
z = pd.read_csv("NeuronProjection2_wPoly.csv", header = None)
cc = []
for i in z[0]:
    cc.append(i.split(" - ")[0])

genes = cc
ra = pd.read_csv("Agoglia_Fraser_2021_DESeq2_HumChp_Hybrid_CS_D100_chpr.txt", sep = "\t")
ra = ra.dropna()
ra = ra[ra["padj"] < 0.1]
rad = ra[ra["log2FoldChange"] < 0]
rau = ra[ra["log2FoldChange"] > 0]
down = rad[rad["Gene"].isin(genes)].shape[0]
up = rau[rau["Gene"].isin(genes)].shape[0]

from scipy.stats import binomtest
binomtest(down, up + down, p = rad.shape[0]/(rad.shape[0] + rau.shape[0]))

In [ ]:
#Plotting cortical organoid data
new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}

sns.set(font_scale = 1.4)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))
#t_ax = sns.barplot({"Upregulated\nin human":5, "Downregulated\nin human":11}, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = {"Upregulated\nin human":new_palette["Human"], "Downregulated\nin human":new_palette["Chimp"]})
t_ax = sns.barplot({"Upregulated\nin human":6, "Downregulated\nin human":16}, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = {"Upregulated\nin human":new_palette["Human"], "Downregulated\nin human":new_palette["Chimp"]})

for patch in t_ax.patches:
    print(patch.get_x() + patch.get_width()/2)
c = 0
for patch in t_ax.patches:
    if c < 1:
        patch.set_edgecolor(new_palette["Human"])
        patch.set_facecolor(new_palette["Human"] + "1A")
    elif c == 1:
        patch.set_edgecolor(new_palette["Chimp"])
        patch.set_facecolor(new_palette["Chimp"] + "1A")
    c += 1
plt.ylabel("Number of genes")
plt.title("Day 100 human-chimp hybrid\ncortical organoids")
plt.ylim(0, 20)

In [ ]:
to_plot = pd.concat([rad[rad['Gene'].isin(genes)], rau[rau['Gene'].isin(genes)]])
to_plot = to_plot.sort_values("log2FoldChange")
out = []
for index, row in to_plot.iterrows():
    #out.append(["$_{" + index + "}$", row["log2FoldChange HumChp"], "Hybrid cortical organoid"])
    #out.append(["$_{" + index + "}$", row["log2FoldChange HumChp Adult"], "Adult MTG L2/3 IT"])
    if row["log2FoldChange"] < 0:
        x = "Down human"
    else:
        x = "Up human"
    out.append(["$_{" + row["Gene"] + "}$", row["log2FoldChange"], x])
to_plot = pd.DataFrame(out)
to_plot.columns = ["Gene", "Log$_{2}$ fold-change", "Direction"]
sns.set(font_scale = 1.5)
sns.set_style("white")
fig, ax = plt.subplots(figsize = (9, 4))
sns.set_style("white")
new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}
t_ax = sns.barplot(data = to_plot, x = "Gene", y = "Log$_{2}$ fold-change", hue = "Direction")


c = 0
for patch in t_ax.patches:
    if c < 16:
        patch.set_edgecolor(new_palette["Chimp"])
        patch.set_facecolor(new_palette["Chimp"] + "1A")
    else:
        patch.set_edgecolor(new_palette["Human"])
        patch.set_facecolor(new_palette["Human"] + "1A")
    c += 1
plt.title("Human-derived ASE in day 100 cortical organoids", size = 15)
plt.xticks(rotation=90)
plt.ylabel("Log$_{2}$ fold-change human/chimp", size = 14)
plt.legend([], [], frameon = False)

In [ ]:
#Now for L2/3 IT neuron data
l23 = pd.read_csv("DESeq2_L2-3_IT_Human_Chimp.txt", sep = "\t")
l23 = l23.dropna()
l23 = l23[l23["padj"] < 0.05]
l23d = l23[l23["log2FoldChange"] < 0]
l23u = l23[l23["log2FoldChange"] > 0]
down = l23d[l23d["Gene"].isin(genes)].shape[0]
up = l23u[l23u["Gene"].isin(genes)].shape[0]

from scipy.stats import binomtest
binomtest(down, up + down, p = l23d.shape[0]/(l23d.shape[0] + l23u.shape[0]))

In [ ]:
new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}

sns.set(font_scale = 1.4)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))
#t_ax = sns.barplot({"Upregulated\nin human":5, "Downregulated\nin human":11}, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = {"Upregulated\nin human":new_palette["Human"], "Downregulated\nin human":new_palette["Chimp"]})
t_ax = sns.barplot({"Upregulated\nin human":8, "Downregulated\nin human":22}, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = {"Upregulated\nin human":new_palette["Human"], "Downregulated\nin human":new_palette["Chimp"]})

for patch in t_ax.patches:
    print(patch.get_x() + patch.get_width()/2)
c = 0
for patch in t_ax.patches:
    if c < 1:
        patch.set_edgecolor(new_palette["Human"])
        patch.set_facecolor(new_palette["Human"] + "1A")
    elif c == 1:
        patch.set_edgecolor(new_palette["Chimp"])
        patch.set_facecolor(new_palette["Chimp"] + "1A")
    c += 1
plt.ylabel("Number of genes")
plt.title("DLPFC L2/3 IT neurons")
plt.ylim(0, 25)

In [ ]:
#Read in data from Velmehsev et al. 2019
c = pd.read_csv("science.adf0834_data_s4.csv", sep = ",")
Counter(c["lineage"])

In [ ]:
#Generally enriched in earlier fetal development for excicatory IT neurons
z = pd.read_csv("NeuronProjection2_wPoly.csv", header = None)
cc = []
for i in z[0]:
    cc.append(i.split(" - ")[0])
from collections import Counter

c = pd.read_csv("science.adf0834_data_s4.csv", sep = ",")
c = c[c["lineage"].isin(["L2_3", "L4", "L5", "L5_6_IT"])]
c_cc = c[c["gene"].isin(cc)].drop_duplicates("gene")
c_ncc = c[(~c["gene"].isin(cc)) & (c["gene"].isin(list(v_mis["Gene"]) + list(vv_mis2["Gene"])))].drop_duplicates("gene")

x = Counter(c_cc["age_max"])
y = Counter(c_ncc["age_max"])

out = []
for key in y.keys():
    try:
        out.append([key, x[key], y[key]])
    except:
        out.append([key, 0, y[key]])
df = pd.DataFrame(out)

#Removing duplicate genes
print(fisher_exact([[17, 1], [141 + 51, 27 + 33 + 1 + 1]]))
#Not removing duplicate genes
print(fisher_exact([[24, 1], [246 + 64, 48 + 34 + 1 + 1]]))

df

In [ ]:
#Generally enriched in earlier fetal development for all neurons, increased power this way

from collections import Counter

c = pd.read_csv("science.adf0834_data_s4.csv", sep = ",")
c = c[c["lineage"].isin(["L2_3", "L4", "L5", "L5_6_IT", "CALB2", "CCK", "NOS", "PV", "PV_MP", "SST", "SST_RELN", "SV2C", "VIP", "L6", "SP"])]
c_cc = c[c["gene"].isin(cc)].drop_duplicates("gene")
c_ncc = c[(~c["gene"].isin(cc)) & (c["gene"].isin(list(v_mis["Gene"]) + list(vv_mis2["Gene"])))].drop_duplicates("gene")

x = Counter(c_cc["age_max"])
y = Counter(c_ncc["age_max"])

out = []
for key in y.keys():
    try:
        out.append([key, x[key], y[key]])
    except:
        out.append([key, 0, y[key]])
df = pd.DataFrame(out)

#Removing duplicate genes
print(fisher_exact([[24, 1], [86 + 186, 50 + 33 + 1 + 1]]))
#Not removing duplicate genes
print(fisher_exact([[48, 4], [514 + 181, 135 + 45 + 1 + 1]]))

df

In [ ]:
#Print list of genes
vk = v_mis[v_mis["Gene"].isin(np.unique(c[(c["gene"].isin(cc)) & (c["age_max"].isin(["2nd trimester", "3rd trimester"]))]["gene"]))]
for i in vk[vk["PhyloP447"] >= 5].sort_values("Gene").drop_duplicates("Gene")["Gene"]:
    print(i)

In [ ]:
genes_t23 = c[(c["gene"].isin(cc)) & (c["age_max"].isin(["2nd trimester", "3rd trimester"]))]["gene"]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
categories = ['Neuron projection genes', 'All genes']
values = [ci_05_proj_02[0], ci_05_all_02[0]]
lower_ci = [ci_05_proj_02[0] - ci_05_proj_02[1], ci_05_all_02[0] - ci_05_all_02[1]]  # Lower confidence intervals

# Create the bar plot
plt.bar(categories, values, color=["#F42FF5", "#40A94D"], label='Values', alpha = 0.8)

# Add lower confidence intervals as error bars
for i, (x, y, ci) in enumerate(zip(categories, values, lower_ci)):
    plt.vlines(x, y - ci, y, color='black', label='Lower CI' if i == 0 else "")
    plt.hlines(y - ci, i-0.125, i+0.125, color='black')
    if i == 0:
        plt.text(i - 0.025, y-0.025, "*", size = 20)

# Customize the plot
plt.xlabel('Gene set', size = 16)
plt.ylabel('Odds ratio', size = 16)
plt.title('MK-Test on nonsynonymous-depleted genes', size = 16)
plt.legend([], [], frameon = False)
plt.xticks(size = 14)
plt.show()



In [ ]:
c[(c["gene"].isin(cc)) & (c["age_max"].isin(["2nd trimester", "3rd trimester"]))]


In [ ]:
#Genes with these developmentally dynamic expression profiles in neurons have especially strong evidence for PS

bleh = c[(c["gene"].isin(cc)) & (c["age_max"].isin(["2nd trimester", "3rd trimester"]))]["gene"]
vx = v_mis[v_mis["Gene"].isin(bleh)]
vvx = vv_mis[vv_mis["Gene"].isin(bleh)]
vvx = vvx[(vvx["UnfoldedMAF"] > 0.5) & (vvx["UnfoldedMAF"] < 0.9)]
vsx = vs[vs["Gene"].isin(bleh)]

vvsx = vvs[vvs["Gene"].isin(bleh)]
vvsx = vvsx[(vvsx["UnfoldedMAF"] > 0.5) & (vvsx["UnfoldedMAF"] < 0.9)]

print([[vx.shape[0], vvx.shape[0]], [vsx.shape[0], vvsx.shape[0]]])
fisher_exact([[vx.shape[0], vvx.shape[0]], [vsx.shape[0], vvsx.shape[0]]])


In [ ]:
#Not so much for other genes
bleh = c[(c["gene"].isin(cc)) & (c["age_max"].isin(["2nd trimester", "3rd trimester"]))]["gene"]
vx = v_mis[v_mis["Gene"].isin(np.setdiff1d(cc, bleh))]
vvx = vv_mis[vv_mis["Gene"].isin(np.setdiff1d(cc, bleh))]
vvx = vvx[(vvx["UnfoldedMAF"] > 0.5) & (vvx["UnfoldedMAF"] < 0.9)]
vsx = vs[vs["Gene"].isin(np.setdiff1d(cc, bleh))]

vvsx = vvs[vvs["Gene"].isin(np.setdiff1d(cc, bleh))]
vvsx = vvsx[(vvsx["UnfoldedMAF"] > 0.5) & (vvsx["UnfoldedMAF"] < 0.9)]

print([[vx.shape[0], vvx.shape[0]], [vsx.shape[0], vvsx.shape[0]]])
fisher_exact([[vx.shape[0], vvx.shape[0]], [vsx.shape[0], vvsx.shape[0]]])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

peak = [7.303051, 1.304627]
npeak = [1.325488, 0.805086]

# Example data
categories = ['No peak', 'Peak in late fetal development']
values = [npeak[0], peak[0]]
lower_ci = [npeak[0] - npeak[1], peak[0] - peak[1]]  # Lower confidence intervals

# Create the bar plot
plt.bar(categories, values, color=["#40A94D", "#F42FF5"], label='Values', alpha = 0.8)

# Add lower confidence intervals as error bars
for i, (x, y, ci) in enumerate(zip(categories, values, lower_ci)):
    plt.vlines(x, y - ci, y, color='black', label='Lower CI' if i == 0 else "")
    plt.hlines(y - ci, i-0.125, i+0.125, color='black')
    #if i == 0:
    #    plt.text(i - 0.025, y-0.025, "*", size = 20)

# Customize the plot
plt.xlabel('Gene set', size = 16)
plt.ylabel('Odds ratio', size = 16)
plt.ylim(0, 8)
plt.title('MK-Test on nonsynonymous-depleted\nneuron projection genes', size = 16)
plt.legend([], [], frameon = False)
plt.xticks(size = 14)
plt.show()

